In [1]:
import scrapy
import json
from scrapy.crawler import CrawlerProcess
class MySpider(scrapy.Spider):
    name = "crawler"
    start_urls = ['https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_INTRADAY&symbol=BTC&market=EUR&apikey=demo']
    
    def parse(self, response):     #Self is passing the object to itself. Response is the website loaded in json.
        #print('test')
        jsonresponse = json.loads(response.body_as_unicode())
    
        yield {
            'Meta Data': jsonresponse["Meta Data"],
            'Time Series (Digital Currency Intraday)': jsonresponse["Time Series (Digital Currency Intraday)"][0]["2018-02-08 11:55:00"]
            }
process = CrawlerProcess({
    'FEED_FORMAT': 'json',
    'FEED_URI': 'PythonLinks3.json',
    # Note that because we are doing API queries, the robots.txt file doesn't apply to us.
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True,
    'LOG_ENABLED': False,
})

In [2]:
# Starting the crawler with our spider.
process.crawl(MySpider)
process.start()
import pandas as pd
test=pd.read_json('PythonLinks3.json', orient = 'records')
print(test['Meta Data'][0])
print(test['Meta Data'][0]['5. Market Name'])

{'1. Information': 'Intraday Prices and Volumes for Digital Currency', '2. Digital Currency Code': 'BTC', '3. Digital Currency Name': 'Bitcoin', '4. Market Code': 'EUR', '5. Market Name': 'Euro', '6. Interval': '5min', '7. Last Refreshed': '2018-02-08 18:30:00', '8. Time Zone': 'UTC'}
Euro


In [8]:
print(test['Time Series (Digital Currency Intraday)'][0]["2018-02-08 11:55:00"])

{'1a. price (EUR)': '6966.24176402', '1b. price (USD)': '8528.41760130', '2. volume': '55427.72191515', '3. market cap (USD)': '472710759.18131000'}


In [15]:
#Extracting all the time stamps.
#test['Time Series (Digital Currency Intraday)'][0].keys()
df = pd.DataFrame({'Time': list(test['Time Series (Digital Currency Intraday)'][0].keys())})
df.head()

,Time
0,2018-02-08 18:30:00
1,2018-02-08 18:25:00
2,2018-02-08 18:20:00
3,2018-02-08 18:15:00
4,2018-02-08 18:10:00


In [22]:
EUR = list()
USD = list()

for time in df['Time']:
    EUR.append(test['Time Series (Digital Currency Intraday)'][0][time]['1a. price (EUR)'])
    USD.append(test['Time Series (Digital Currency Intraday)'][0][time]['1b. price (USD)'])

In [23]:
df['EUR_px'] = EUR
df['USD_px'] = USD
print(df.head())

                  Time         EUR_px         USD_px
0  2018-02-08 18:30:00  6648.68417241  8144.80304347
1  2018-02-08 18:25:00  6666.16796579  8166.22112407
2  2018-02-08 18:20:00  6656.55212047  8154.44147502
3  2018-02-08 18:15:00  6631.15257928  8123.32640698
4  2018-02-08 18:10:00  6619.34217506  8108.85836883


In [24]:
df2 = pd.DataFrame({'Meta_Data': list(test['Meta Data'][0].keys())})
df2.head()

,Meta_Data
0,1. Information
1,2. Digital Currency Code
2,3. Digital Currency Name
3,4. Market Code
4,5. Market Name


In [30]:
for row in range(df2.shape[0]):
    df2.loc[row,'Meta_Data'] = ''.join([i for i in df2.loc[row, 'Meta_Data'] if (not i.isdigit() and i != '.')])
print(df2.head())

                Meta_Data
0             Information
1   Digital Currency Code
2   Digital Currency Name
3             Market Code
4             Market Name


In [31]:
test['Meta Data'][0].keys()

dict_keys(['1. Information', '2. Digital Currency Code', '3. Digital Currency Name', '4. Market Code', '5. Market Name', '6. Interval', '7. Last Refreshed', '8. Time Zone'])

In [32]:
test['Meta Data'][0].values()

dict_values(['Intraday Prices and Volumes for Digital Currency', 'BTC', 'Bitcoin', 'EUR', 'Euro', '5min', '2018-02-08 18:30:00', 'UTC'])